In [28]:
%matplotlib inline
import os
import numpy as np
import scipy as sp
import datetime
import matplotlib.pyplot as plt
from scipy.integrate import odeint 
import pandas as pd
import scipy as sp 


In [92]:
h = ['claimID', 'claim', 'label', 'claimURL', 'reason', 'categories', 'speaker', 'checker', 'tags', 'articleTitle', 'publishDate', 'claimDate', 'entities']

df_train = pd.read_csv('../data/train.tsv',sep='\t',names=h,header=None)
df_train.set_index('claimID',inplace = True)
df_train.head()

,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
claimID,,,,,,,,,,,,
pomt-03627,Six out of 10 of the highest unemployment rate...,half-true,/ohio/statements/2013/may/06/chris-redfern/ohi...,When a couple of Statehouse Republicans prepar...,None,Chris Redfern,None,None,None,2013-05-06T06:00:00,2013-04-30,['None']
pomt-09611,No Democratic campaign for (Fla.) governor has...,true,/florida/statements/2010/jan/15/alex-sink/flor...,Florida's leading Republican candidate for gov...,None,Alex Sink,None,None,None,2010-01-15T13:59:00,2010-01-06,['None']
tron-00214,Forward an email for Jasmine,fiction!,https://www.truthorfiction.com/jasmine/,None,9-11-attack,None,None,None,Forward an email for Jasmine,"Mar 17, 2015",None,['None']
snes-04484,Pope Francis endorsed Donald Trump for president.,false,https://www.snopes.com/fact-check/pope-francis...,None,Junk News,None,Dan Evon,None,"Pope Francis Shocks World, Endorses Donald Tru...",10 July 2016,None,['None']
pomt-06704,Says Ron Paul insisted FEMA should be shut down.,true,/texas/statements/2011/sep/03/maureen-dowd/mau...,Commenting on the federal response to Hurrican...,None,Maureen Dowd,None,None,None,2011-09-03T06:00:00,2011-08-30,['None']


In [26]:
h2 = ['claimID', 'claim', 'claimURL',' reason', 'categories', 'speaker', 'checker',' tags', 'articleTitle', 'publishDate', 'claimDate', 'entities']

df_test = pd.read_csv('../data/test.tsv',sep='\t',names=h2,header=None)
df_test.set_index('claimID',inplace = True)
df_test.head()

,claim,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
claimID,,,,,,,,,,,
pomt-14759,"In 2000, ""I wrote about Osama bin Laden, ‘We’v...",/virginia/statements/2015/dec/11/donald-trump/...,Almost two years before the Sept. 11 terror at...,None,Donald Trump,None,None,None,2015-12-11T10:44:11,2015-12-02,['Osama_bin_Laden']
pomt-02596,"Says in 2000, Fox News broke the story of Geor...",/punditfact/statements/2014/jan/27/ann-coulter...,"In the world of cable television news, most pe...",None,Ann Coulter,None,None,None,2014-01-27T16:20:31,2014-01-22,"['Fox_News_Channel', 'George_W._Bush']"
snes-00569,A video shows a powerful jet of water flipping...,https://www.snopes.com/fact-check/child-flippe...,None,Fauxtography,None,Dan Evon,None,Does This Video Show a Child Flipped by a Foun...,21 May 2018,None,['None']
pomt-14237,My numbers are better right now than Ronald Re...,/truth-o-meter/statements/2016/apr/14/donald-t...,"During a rally in Rome, N.Y., Republican presi...",None,Donald Trump,None,None,None,2016-04-14T10:05:57,2016-04-11,"['Jimmy_Carter', 'Ronald_Reagan']"
snes-05947,The U.S. Congress is considering passage of th...,https://www.snopes.com/fact-check/americans-no...,None,Uncategorized,None,David Mikkelson,None,Americans with No Abilities Act,23 June 2006,None,"['United_States_Congress', 'United_States']"


In [27]:
df_train.label.value_counts()

false                              5537
mostly true                        2466
mostly false                       2454
true                               2442
half-true                          2428
0                                  1957
pants on fire!                     1287
fiction!                           1246
none                                933
truth!                              621
mixture                             555
promise kept                        441
unproven                            431
facebook scams                      384
1                                   253
true messages                       242
promise broken                      218
compromise                          185
unproven!                           176
bogus warning                       176
in-between                          175
truth! & fiction!                   137
verdict: true                       125
verdict: false                      124
miscaptioned                        122


In [119]:
h = ['snip_id', 'snip_1', 'snip_2','snip_url']
df_snip =  pd.DataFrame(columns=h)


dir = '..\d'
for filename in os.listdir(dir):
    
    df_row = pd.read_csv(os.path.join(dir, filename),names=h,sep='\t',header=None)
    df_row.insert(loc = 0, column = 'claimID', value = filename )
    
    df_snip = df_snip.append(df_row)
 
    
  
df_snip.set_index(['claimID','snip_id'],inplace = True)
df_snip.sort_index(inplace = True)

df_snip    
    

snip_1  \
claimID    snip_id                                                      
abbc-00002 2        Full text: Senator Fraser Anning's maiden spee...   
           3        Fraser Anning speech: Senator talks of final s...   
           4                      Parliament of Australia - Wikipedia   
           5        Fraser Anning's Speech To Australia's House Of...   
           6        Australian politician condemned for using infa...   
           7        In case you missed it | Learning | Parliamenta...   
           8                                     Senator Claire Moore   
           9        Latest and Breaking News | The Canberra Times ...   
           10       Brisbane Times - Brisbane News, Queensland New...   
abbc-00003 2        Australian senator calls for 'final solution t...   
           3        Education Talking Points | 24/08/18 - Austcham...   
           4        Full text: Senator Fraser Anning's maiden spee...   
           5        Fact Check: Would reducing international stude...   
           6        Australian Senator Fraser Anning Compares Musl...   
           7        Bob Katter's vicious ram raid on One Nation - ...   
           8        Social Sciences | Free Full-Text | Islamophobi...   
           9        Bob Katter accuses a journalist of being racis...   
           10       Senator F. Anning Hugely Offends PC Racist Mai...   
abbc-00004 2        Fact check: Are Muslims in NSW and Victoria th...   
           3        Fraser Anning's call for Muslim migration ban ...   
           4        Minor parties - ABC News (Australian Broadcast...   
           5        Mehreen Faruqi Of Pakistani-Origin Becomes Fir...   
           6        Bob Katter accuses a journalist of being racis...   
           7        Hanson says Anning speech went too far, but wa...   
           8        Anti-Muslim senator Fraser Anning is kicked ou...   
           9        United Patriots Front: St Kilda Beach, Melbour...   
           10       Fraser Anning defends his appearance at United...   
abbc-00005 2        Are more than half of Australia's working-age ...   
           3        Fact check: Are more than half of Australia's ...   
           4        Media “fact checks” Senator who said more than...   
           5        Australian senator Fraser Anning defends 'neo-...   
           6        Full text: Senator Fraser Anning's maiden spee...   
           7        Manus Island violence: shocking but not surpri...   
           8        Confronting the Return of Race Politics | Aust...   
           9        Where is the alternative to Manus Island cruel...   
           10                                      Australian history   

                                                               snip_2  \
claimID    snip_id                                                      
abbc-00002 2        Aug 15, 2018 ... Katter's Australia Party sena...   
           3        Aug 15, 2018 ... Queensland Senator Fraser Ann...   
           4        The Parliament of Australia is the legislative...   
           5        Aug 19, 2018 ... I am pleased to advise that t...   
           6        Aug 15, 2018 ... It was one of the most divisi...   
           7        A number of private senators' and members' bil...   
           8        Aug 17, 2018 ... Authorised by Senator Claire ...   
           9        Canberra light rail workers on shift in near-4...   
           10       Hutchison Ports is locked in a fresh dispute w...   
abbc-00003 2        Aug 14, 2018 ... Crossbencher Fraser Anning wi...   
           3        Aug 24, 2018 ... Australia is new destination ...   
           4        Aug 15, 2018 ... Katter's Australia Party sena...   
           5        Aug 27, 2018 ... Would reducing international ...   
           6        Aug 15, 2018 ... Home | World ... Australian S...   
           7        Aug 16, 2018 ... “The final solution to the im...   
           8        Oct 30, 2018 ... Among the 

In [142]:
df_train.loc['abbc-00002']['claim']

'In his first speech to Parliament, Senator Fraser Anning, a member of Katter\'s Australian Party, made a number of disputed claims including that "a majority of Muslims in Australia of working age do not work and live on welfare".'

In [141]:


print(df_train.loc['abbc-00002']['claim'])

df_snip.loc['abbc-00002'].loc[2]['snip_1']

In his first speech to Parliament, Senator Fraser Anning, a member of Katter's Australian Party, made a number of disputed claims including that "a majority of Muslims in Australia of working age do not work and live on welfare".


"Full text: Senator Fraser Anning's maiden speech | SBS News"

In [ ]:
 df_train.loc[filename,'snip_id' ] = str(df_row['id'].tolist())
    df_train.loc[filename,'snip_1' ] = str(df_row['1'].tolist())
    df_train.loc[filename,'snip_2' ] = str(df_row['2'].tolist())
    df_train.loc[filename,'snip_url' ] = str(df_row['url'].tolist())